<H2>Prerequisite</H2>

* pip install pmdarima

In [1]:
import numpy as np
import pandas as pd
import pmdarima as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [10]:
df = pd.read_csv("/Users/keremsmacbook/DataspellProjects/MultivariateForecasting/Data/SP500_Cleaned.csv")

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [14]:
df.shape

(2764, 461)

In [15]:
df.head(10)

,Unnamed: 0,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,0,27.283405,13.179525,16.769093,22.419830,26.000973,14.793333,56.624992,38.340000,34.286098,...,19.033312,95.529976,19.385836,56.165749,37.213242,23.749273,39.127857,60.207806,40.959999,17.587208
2013-01-03,1,27.381128,12.877850,16.557428,22.234699,26.990719,14.750000,56.419998,37.750000,33.732830,...,19.033312,96.557526,19.329191,56.064457,37.112217,23.671537,39.372551,61.070477,41.000000,17.611206
2013-01-04,2,27.921843,13.886581,16.096222,21.953815,26.828459,14.876667,56.731575,38.130001,33.132832,...,19.190020,97.722618,19.400000,56.324043,37.433651,23.637007,39.803684,61.381714,40.669998,18.115057
2013-01-07,3,27.719885,13.990280,16.001547,21.998503,27.047497,14.730000,56.485600,37.939999,33.234123,...,19.248785,98.054352,19.194668,55.671913,37.562252,23.326099,39.553158,61.532909,40.900002,18.027084
2013-01-08,4,27.498386,14.291961,16.044611,21.519714,27.055613,14.750000,56.813564,38.139999,32.891258,...,19.379377,97.973450,19.230074,56.020145,37.130569,23.041113,37.892735,61.621864,40.930000,17.667183
2013-01-09,5,28.241058,14.263677,15.793853,21.641014,27.234089,14.803333,57.215347,38.660000,32.805550,...,19.823374,98.321350,19.258394,55.804855,37.112217,23.403828,37.960068,62.760197,41.369999,17.475239
2013-01-10,6,28.449533,14.273106,15.989633,21.704851,27.461245,14.836667,56.715172,38.619999,33.202961,...,19.940905,99.437920,19.258394,56.412678,37.038750,23.731987,38.797516,62.884705,41.150002,17.555218
2013-01-11,7,28.293179,13.933719,15.891588,21.866341,27.184271,14.926667,57.297321,38.090000,32.930241,...,20.110674,99.510735,19.237150,56.735592,36.671391,23.472908,39.160606,63.293781,41.299999,17.395258
2013-01-14,8,28.364840,13.830017,15.325013,22.021378,27.176128,15.026667,57.280918,38.160000,32.914646,...,20.110674,99.356995,19.173429,56.716572,36.561184,23.498819,38.738945,63.187073,40.980000,17.483234


In [19]:
df.drop("Unnamed: 0", inplace = True, axis=1)

In [20]:
df.head(10)

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,27.283405,13.179525,16.769093,22.419830,26.000973,14.793333,56.624992,38.340000,34.286098,21.421146,...,19.033312,95.529976,19.385836,56.165749,37.213242,23.749273,39.127857,60.207806,40.959999,17.587208
2013-01-03,27.381128,12.877850,16.557428,22.234699,26.990719,14.750000,56.419998,37.750000,33.732830,21.249302,...,19.033312,96.557526,19.329191,56.064457,37.112217,23.671537,39.372551,61.070477,41.000000,17.611206
2013-01-04,27.921843,13.886581,16.096222,21.953815,26.828459,14.876667,56.731575,38.130001,33.132832,21.832092,...,19.190020,97.722618,19.400000,56.324043,37.433651,23.637007,39.803684,61.381714,40.669998,18.115057
2013-01-07,27.719885,13.990280,16.001547,21.998503,27.047497,14.730000,56.485600,37.939999,33.234123,20.928026,...,19.248785,98.054352,19.194668,55.671913,37.562252,23.326099,39.553158,61.532909,40.900002,18.027084
2013-01-08,27.498386,14.291961,16.044611,21.519714,27.055613,14.750000,56.813564,38.139999,32.891258,21.174585,...,19.379377,97.973450,19.230074,56.020145,37.130569,23.041113,37.892735,61.621864,40.930000,17.667183
2013-01-09,28.241058,14.263677,15.793853,21.641014,27.234089,14.803333,57.215347,38.660000,32.805550,21.279190,...,19.823374,98.321350,19.258394,55.804855,37.112217,23.403828,37.960068,62.760197,41.369999,17.475239
2013-01-10,28.449533,14.273106,15.989633,21.704851,27.461245,14.836667,56.715172,38.619999,33.202961,21.174585,...,19.940905,99.437920,19.258394,56.412678,37.038750,23.731987,38.797516,62.884705,41.150002,17.555218
2013-01-11,28.293179,13.933719,15.891588,21.866341,27.184271,14.926667,57.297321,38.090000,32.930241,21.174585,...,20.110674,99.510735,19.237150,56.735592,36.671391,23.472908,39.160606,63.293781,41.299999,17.395258
2013-01-14,28.364840,13.830017,15.325013,22.021378,27.176128,15.026667,57.280918,38.160000,32.914646,21.338957,...,20.110674,99.356995,19.173429,56.716572,36.561184,23.498819,38.738945,63.187073,40.980000,17.483234


In [21]:
def inverse_difference(original_series, differences, interval=1):
    return original_series + differences.cumsum()

def inverse_log(x):
    return np.exp(x)

In [22]:
results = []

In [23]:
for company in df.columns:

    c = pd.DataFrame(df[company])

    split_size = int(0.8 * len(c))
    train = c.iloc[:split_size]
    test = c.iloc[split_size:]

    train_log = np.log(train)
    train_log_diff = train_log.diff().dropna()
    
    auto_arima_model_3 = pm.auto_arima(train_log_diff[company], start_p=1, start_q=1,
                                       test='adf',
                                       max_p=5, max_q=5,
                                       m=1,
                                       d=None,
                                       seasonal=False,
                                       start_P=0,
                                       D=0,
                                       trace=False,
                                       error_action='ignore',
                                       suppress_warnings=True,
                                       stepwise=True)

    forecast = auto_arima_model_3.predict(n_periods=len(test))

    forecast_df = pd.DataFrame(forecast, columns=[company])
    forecast_df.index = test.index

    forecast_df = pd.DataFrame(forecast, columns=[company])
    forecast_df.index = test.index

    forecast_df_cumsum = forecast_df[company].cumsum()
    last_log_value = train_log.iloc[-1].item()  # Convert to scalar if necessary
    forecast_df_reversed_diff = forecast_df_cumsum + last_log_value
    forecast_df_reversed = np.exp(forecast_df_reversed_diff)

    forecasted_df = pd.DataFrame(forecast_df_reversed, columns=[company])
    forecasted_df.index = test.index

    forecasted_values = forecasted_df[company].values
    actual_values = test[company].values


    rmse = np.sqrt(mean_squared_error(actual_values, forecasted_values))


    mae = mean_absolute_error(actual_values, forecasted_values)


    mape = np.mean(np.abs((actual_values - forecasted_values) / actual_values)) * 100

    print(company, rmse, mae, mape)
    results.append([company, rmse, mae, mape])

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


A 58.07333519023978 50.10532095354031 40.25930112687721


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


AAL 5.494589174489216 5.013516163263797 35.59011419739155


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


...

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


ZBH 19.18048924273016 16.923203025114233 14.792746099203107


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


ZBRA 413.73957671722843 365.83547376544624 133.4501471997249
ZION 16.88296246365082 13.445006372268987 38.892462673108255


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [24]:
results_df = pd.DataFrame(results, columns=['Company Name', 'RMSE', 'MAE', 'MAPE'])

In [25]:
results_df.to_excel('/Users/keremsmacbook/DataspellProjects/MultivariateForecasting/OutputData/S&P500_NBeats.xlsx', index=False)